In [ ]:
!pip install gdown==4.6.0

  Attempting uninstall: gdown
    Found existing installation: gdown 5.2.0
    Uninstalling gdown-5.2.0:
      Successfully uninstalled gdown-5.2.0


Data google drive link của Hùng


In [4]:
import gdown
!mkdir -p content/data
# Download all files
file_ids = [
    '1JXdyh-UVveXhBToLJj8ZJNttiJif6zH8',  # X_train
    '1rtyLyNDBr3sPcyElMhpDPK3qTbNZH1YT',  # X_test
    '1IkNvEhPFfano0qSDZFWhsmpF3ttC1q0Y',  # y_train
    '1JlMWumYQP9OIH79dVsCiZ-eavZAhgSzv'   # y_test
]

file_paths = [
    'content/data/X_train.csv',
    'content/data/X_test.csv',
    'content/data/y_train.csv',
    'content/data/y_test.csv'
]
for file_id, file_path in zip(file_ids, file_paths):
    gdown.download(f'https://drive.google.com/uc?id={file_id}', file_path, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1JXdyh-UVveXhBToLJj8ZJNttiJif6zH8
To: /content/content/data/X_train.csv
100%|██████████| 5.03M/5.03M [00:00<00:00, 64.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rtyLyNDBr3sPcyElMhpDPK3qTbNZH1YT
To: /content/content/data/X_test.csv
100%|██████████| 1.34M/1.34M [00:00<00:00, 20.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1IkNvEhPFfano0qSDZFWhsmpF3ttC1q0Y
To: /content/content/data/y_train.csv
100%|██████████| 16.9k/16.9k [00:00<00:00, 25.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1JlMWumYQP9OIH79dVsCiZ-eavZAhgSzv
To: /content/content/data/y_test.csv
100%|██████████| 4.24k/4.24k [00:00<00:00, 6.96MB/s]


Kiểm tra data

In [5]:
# Đọc file CSV
import pandas as pd
df_x_train = pd.read_csv('content/data/X_train.csv')
df_x_test = pd.read_csv('content/data/X_test.csv')
df_y_train = pd.read_csv('content/data/y_train.csv')
df_y_test = pd.read_csv('content/data/y_test.csv')

# Hiển thị thông tin và cột
print("\n🟦 X_train:")
print(df_x_train.info())
print("Tên cột:", df_x_train.columns.tolist())

print("\n🟨 X_test:")
print(df_x_test.info())
print("Tên cột:", df_x_test.columns.tolist())

print("\n🟩 y_train:")
print(df_y_train.info())
print("Tên cột:", df_y_train.columns.tolist())

print("\n🟥 y_test:")
print(df_y_test.info())
print("Tên cột:", df_y_test.columns.tolist())



🟦 X_train:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8457 entries, 0 to 8456
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   message  8445 non-null   object
dtypes: object(1)
memory usage: 66.2+ KB
None
Tên cột: ['message']

🟨 X_test:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2115 entries, 0 to 2114
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   message  2114 non-null   object
dtypes: object(1)
memory usage: 16.7+ KB
None
Tên cột: ['message']

🟩 y_train:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8457 entries, 0 to 8456
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   category  8457 non-null   int64
dtypes: int64(1)
memory usage: 66.2 KB
None
Tên cột: ['category']

🟥 y_test:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2115 entries, 0 to 2114
Data columns (total 1 columns

BOWs

In [9]:
# 1. Import các thư viện cần thiết
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer  # Thay đổi từ TfidfVectorizer sang CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
import string
import re

# 2. Tải dữ liệu từ nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# 3. Đọc dữ liệu từ file CSV
# Chú ý cập nhật đường dẫn đúng nếu sử dụng file của bạn
# df_x_train = pd.read_csv('path_to_X_train.csv')
# df_x_test = pd.read_csv('path_to_X_test.csv')
# df_y_train = pd.read_csv('path_to_y_train.csv')
# df_y_test = pd.read_csv('path_to_y_test.csv')

# 4. Xử lý dữ liệu đầu vào
# Làm sạch dữ liệu
df_x_train['message'] = df_x_train['message'].fillna('')  # Xử lý NaN
df_x_test['message'] = df_x_test['message'].fillna('')

X_raw = df_x_train['message']
y = df_y_train['category']

# 5. Mã hóa nhãn
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# 6. Hàm tiền xử lý văn bản nâng cao
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()  # Lowercasing
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    tokens = word_tokenize(text)  # Tokenize
    lemmatized = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]  # Lemmatization & remove non-alpha
    return ' '.join(lemmatized)

df_x_train['message'] = df_x_train['message'].apply(preprocess)
df_x_test['message'] = df_x_test['message'].apply(preprocess)

# 7. Chuyển sang Bag of Words (BOW)
vectorizer = CountVectorizer(max_features=17, stop_words='english')  # Sử dụng CountVectorizer và chọn số chiều là 17
X_train_bow = vectorizer.fit_transform(df_x_train['message'])
X_test_bow = vectorizer.transform(df_x_test['message'])

# 8. Scale dữ liệu
scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(X_train_bow)
X_test_scaled = scaler.transform(X_test_bow)

# 9. Huấn luyện mô hình KNN với khoảng cách Cosine và số hàng xóm = 17
param_grid = {
    'n_neighbors': [17],  # Chọn số hàng xóm = 17
    'metric': ['cosine']  # Khoảng cách cosine
}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid.fit(X_train_scaled, y_encoded)

# 10. Dự đoán và đánh giá kết quả
y_pred = grid.best_estimator_.predict(X_test_scaled)
print("Best KNN Parameters:", grid.best_params_)
print("Accuracy:", accuracy_score(df_y_test['category'], y_pred))
print("Classification Report:\n", classification_report(le.inverse_transform(df_y_test['category']), le.inverse_transform(y_pred)))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Best KNN Parameters: {'metric': 'cosine', 'n_neighbors': 17}
Accuracy: 0.8028368794326242
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.95      0.88      1566
           1       0.74      0.38      0.50       549

    accuracy                           0.80      2115
   macro avg       0.77      0.66      0.69      2115
weighted avg       0.79      0.80      0.78      2115



Với TF-IDF, ta chọn các siêu tham số k, khoảng cách, số chiều vector

In [ ]:
# 1. Import các thư viện cần thiết
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
import string
import re

# 2. Tải dữ liệu từ nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# 3. Đọc dữ liệu từ file CSV
# Chú ý cập nhật đường dẫn đúng nếu sử dụng file của bạn
# df_x_train = pd.read_csv('path_to_X_train.csv')
# df_x_test = pd.read_csv('path_to_X_test.csv')
# df_y_train = pd.read_csv('path_to_y_train.csv')
# df_y_test = pd.read_csv('path_to_y_test.csv')

# 4. Xử lý dữ liệu đầu vào
# Làm sạch dữ liệu
df_x_train['message'] = df_x_train['message'].fillna('')  # Xử lý NaN
df_x_test['message'] = df_x_test['message'].fillna('')

X_raw = df_x_train['message']
y = df_y_train['category']

# 5. Mã hóa nhãn
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# 6. Hàm tiền xử lý văn bản nâng cao
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()  # Lowercasing
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    tokens = word_tokenize(text)  # Tokenize
    lemmatized = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]  # Lemmatization & remove non-alpha
    return ' '.join(lemmatized)

df_x_train['message'] = df_x_train['message'].apply(preprocess)
df_x_test['message'] = df_x_test['message'].apply(preprocess)

# 7. So sánh các kích thước TF-IDF khác nhau và chọn số chiều tốt nhất
feature_sizes = [3000, 4000, 5000]  # Các số chiều khác nhau để thử
best_accuracy = 0
best_max_feat = 0

for max_feat in feature_sizes:
    # TF-IDF vectorization với 3-grams
    vectorizer = TfidfVectorizer(max_features=max_feat, stop_words='english', ngram_range=(1, 3))
    X_tfidf = vectorizer.fit_transform(df_x_train['message'])

    # Scale dữ liệu
    scaler = StandardScaler(with_mean=False)
    X_train_scaled = scaler.fit_transform(X_tfidf)
    X_test_tfidf = vectorizer.transform(df_x_test['message'])
    X_test_scaled = scaler.transform(X_test_tfidf)

    # GridSearch để tìm KNN tốt nhất
    param_grid = {
        'n_neighbors': [5, 9, 13, 17],
        'metric': ['euclidean', 'cosine']
    }

    grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=3, scoring='accuracy', n_jobs=-1)
    grid.fit(X_train_scaled, y_encoded)

    # Dự đoán và tính accuracy
    y_pred = grid.best_estimator_.predict(X_test_scaled)
    accuracy = accuracy_score(df_y_test['category'], le.inverse_transform(y_pred))

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_max_feat = max_feat

# Sau khi loop qua tất cả các số chiều, in ra kết quả
print(f"\n==================== Best TF-IDF max_features={best_max_feat} with Accuracy={best_accuracy} ====================")

# Dùng số chiều tốt nhất để huấn luyện lại mô hình
vectorizer = TfidfVectorizer(max_features=best_max_feat, stop_words='english', ngram_range=(1, 3))
X_tfidf = vectorizer.fit_transform(df_x_train['message'])

# Scale dữ liệu
scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(X_tfidf)
X_test_tfidf = vectorizer.transform(df_x_test['message'])
X_test_scaled = scaler.transform(X_test_tfidf)

# Huấn luyện lại KNN với số chiều tốt nhất
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid.fit(X_train_scaled, y_encoded)

# In kết quả cuối cùng
y_pred = grid.best_estimator_.predict(X_test_scaled)
print("Best KNN Parameters:", grid.best_params_)
print("Accuracy:", accuracy_score(df_y_test['category'], y_pred))
print("Classification Report:\n", classification_report(le.inverse_transform(df_y_test['category']), le.inverse_transform(y_pred)))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!



==================== Best TF-IDF max_features=5000 with Accuracy=0.8959810874704491 ====================
Best KNN Parameters: {'metric': 'cosine', 'n_neighbors': 17}
Accuracy: 0.8959810874704491
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.96      0.93      1566
           1       0.86      0.72      0.78       549

    accuracy                           0.90      2115
   macro avg       0.88      0.84      0.86      2115
weighted avg       0.89      0.90      0.89      2115



Với Word embedding: Word2vec

TF-IDF+ SentenceStransformer+PCA

In [ ]:
# 1. Import các thư viện cần thiết
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
import string
import re
from sentence_transformers import SentenceTransformer
from scipy.sparse import csr_matrix, hstack
from sklearn.decomposition import PCA
import logging

# 2. Tắt warning không cần thiết
logging.getLogger("huggingface_hub").setLevel(logging.ERROR)
logging.getLogger("sentence_transformers.SentenceTransformer").setLevel(logging.ERROR)

# 3. Tải dữ liệu từ nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# 4. Đọc dữ liệu từ file CSV
# df_x_train = pd.read_csv('path_to_X_train.csv')
# df_x_test = pd.read_csv('path_to_X_test.csv')
# df_y_train = pd.read_csv('path_to_y_train.csv')
# df_y_test = pd.read_csv('path_to_y_test.csv')

# 5. Xử lý dữ liệu đầu vào
df_x_train['message'] = df_x_train['message'].fillna('')
df_x_test['message'] = df_x_test['message'].fillna('')
X_raw = df_x_train['message']
y = df_y_train['category']

# 6. Mã hóa nhãn
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# 7. Hàm tiền xử lý văn bản nâng cao
lemmatizer = WordNetLemmatizer()
def preprocess(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    lemmatized = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]
    return ' '.join(lemmatized)

df_x_train['message'] = df_x_train['message'].apply(preprocess)
df_x_test['message'] = df_x_test['message'].apply(preprocess)

# 8. Tải mô hình embedding
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# 9. Chọn giá trị max_features
max_feat = 3000  # hoặc giá trị bạn muốn
param_grid = {
    'n_neighbors': [17],
    'metric': ['cosine']
}

# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=max_feat, stop_words='english', ngram_range=(1, 3))
X_tfidf = vectorizer.fit_transform(df_x_train['message'])

# Sentence embedding
X_embed = embed_model.encode(df_x_train['message'], show_progress_bar=False)
X_embed_sparse = csr_matrix(X_embed)

# Combine TF-IDF + Embedding
X_combined = hstack([X_tfidf, X_embed_sparse])

# Test data
X_test_tfidf = vectorizer.transform(df_x_test['message'])
X_test_embed = embed_model.encode(df_x_test['message'], show_progress_bar=False)
X_test_embed_sparse = csr_matrix(X_test_embed)
X_test_combined = hstack([X_test_tfidf, X_test_embed_sparse])

# Scale dữ liệu
scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(X_combined)
X_test_scaled = scaler.transform(X_test_combined)

# PCA giảm chiều xuống 200 chiều
pca = PCA(n_components=200)
X_train_pca = pca.fit_transform(X_train_scaled.toarray())
X_test_pca = pca.transform(X_test_scaled.toarray())

# Grid search
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid.fit(X_train_pca, y_encoded)

y_pred = grid.best_estimator_.predict(X_test_pca)

print("Best KNN Parameters:", grid.best_params_)
print("Accuracy:", accuracy_score(df_y_test['category'], y_pred))
print("Classification Report:\n", classification_report(le.inverse_transform(df_y_test['category']), le.inverse_transform(y_pred)))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Best KNN Parameters: {'metric': 'cosine', 'n_neighbors': 17}
Accuracy: 0.9361702127659575
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.95      0.96      1566
           1       0.87      0.89      0.88       549

    accuracy                           0.94      2115
   macro avg       0.92      0.92      0.92      2115
weighted avg       0.94      0.94      0.94      2115



Fastext Embedding

In [1]:
import gensim.downloader as api

# Tải mô hình fasttext tiếng Anh (wiki-news-300d-1M)
fasttext_model = api.load('fasttext-wiki-news-subwords-300')

[==================================================] 100.0% 958.5/958.4MB downloaded


In [8]:
# 1. Import thư viện
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import re
import gensim.downloader as api

# 2. Tắt warning
import logging
logging.getLogger("gensim").setLevel(logging.ERROR)
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# 3. Đọc dữ liệu
# df_x_train = pd.read_csv('path_to_X_train.csv')
# df_x_test = pd.read_csv('path_to_X_test.csv')
# df_y_train = pd.read_csv('path_to_y_train.csv')
# df_y_test = pd.read_csv('path_to_y_test.csv')

# Giả sử bạn đã có dữ liệu df_x_train, df_x_test, df_y_train, df_y_test

# 4. Tiền xử lý văn bản
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()  # Chuyển tất cả thành chữ thường
    text = re.sub(r'\d+', '', text)  # Xóa số
    text = text.translate(str.maketrans('', '', string.punctuation))  # Xóa dấu câu
    tokens = word_tokenize(text)  # Tách từ
    lemmatized = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]  # Lemmatization
    return ' '.join(lemmatized)  # Trả về văn bản đã xử lý

# Áp dụng tiền xử lý cho tập huấn luyện và kiểm tra
df_x_train['message'] = df_x_train['message'].fillna('').apply(preprocess)
df_x_test['message'] = df_x_test['message'].fillna('').apply(preprocess)

# 5. Mã hóa nhãn
le = LabelEncoder()
y_encoded = le.fit_transform(df_y_train['category'])

# 6. Tải mô hình FastText từ gensim
fasttext_model = api.load('fasttext-wiki-news-subwords-300')

# 7. Vector hóa văn bản với FastText
def get_ft_vector(text):
    # Tokenize câu
    tokens = word_tokenize(text)

    # Lấy vector của từng từ và tính trung bình
    word_vectors = []
    for token in tokens:
        if token in fasttext_model.key_to_index:  # Kiểm tra từ có trong mô hình không
            word_vectors.append(fasttext_model[token])  # Lấy vector của từ

    # Tính trung bình vector các từ trong câu
    if word_vectors:
        return np.mean(word_vectors, axis=0)  # Trung bình của các vector từ
    else:
        # Nếu không có từ nào trong mô hình thì trả về vector 0
        return np.zeros(fasttext_model.vector_size)

# Áp dụng cho tập huấn luyện và kiểm tra
X_train_ft = np.array([get_ft_vector(text) for text in df_x_train['message']])
X_test_ft = np.array([get_ft_vector(text) for text in df_x_test['message']])

# 8. Chuẩn hóa đặc trưng
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_ft)  # Chuẩn hóa cho tập huấn luyện
X_test_scaled = scaler.transform(X_test_ft)  # Chuẩn hóa cho tập kiểm tra

# 9. Huấn luyện mô hình KNN
param_grid = {
    'n_neighbors': [17],
    'metric': ['cosine']
}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid.fit(X_train_scaled, y_encoded)

# 10. Dự đoán và đánh giá kết quả
y_pred = grid.best_estimator_.predict(X_test_scaled)

# 11. Đánh giá mô hình
print("Best KNN Parameters:", grid.best_params_)
print("Accuracy:", accuracy_score(df_y_test['category'], le.inverse_transform(y_pred)))
print("Classification Report:\n", classification_report(le.inverse_transform(df_y_test['category']), le.inverse_transform(y_pred)))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Best KNN Parameters: {'metric': 'cosine', 'n_neighbors': 17}
Accuracy: 0.9030732860520094
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.91      0.93      1566
           1       0.78      0.87      0.82       549

    accuracy                           0.90      2115
   macro avg       0.87      0.89      0.88      2115
weighted avg       0.91      0.90      0.90      2115



KNN + Word2Vec

In [11]:
# 1. Import thư viện
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import re
import gensim.downloader as api

# 2. Tắt warning
import logging
logging.getLogger("gensim").setLevel(logging.ERROR)
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# 3. Đọc dữ liệu
# df_x_train = pd.read_csv('path_to_X_train.csv')
# df_x_test = pd.read_csv('path_to_X_test.csv')
# df_y_train = pd.read_csv('path_to_y_train.csv')
# df_y_test = pd.read_csv('path_to_y_test.csv')

# Giả sử bạn đã có dữ liệu df_x_train, df_x_test, df_y_train, df_y_test

# 4. Tiền xử lý văn bản
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    lemmatized = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]
    return ' '.join(lemmatized)

# Áp dụng tiền xử lý cho tập huấn luyện và kiểm tra
df_x_train['message'] = df_x_train['message'].fillna('').apply(preprocess)
df_x_test['message'] = df_x_test['message'].fillna('').apply(preprocess)

# 5. Mã hóa nhãn
le = LabelEncoder()
y_encoded = le.fit_transform(df_y_train['category'])

# 6. Tải mô hình Word2Vec từ gensim
word2vec_model = api.load('word2vec-google-news-300')

# 7. Vector hóa văn bản với Word2Vec
def get_w2v_vector(text):
    tokens = word_tokenize(text)
    word_vectors = [word2vec_model[token] for token in tokens if token in word2vec_model.key_to_index]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

# Áp dụng cho tập huấn luyện và kiểm tra
X_train_w2v = np.array([get_w2v_vector(text) for text in df_x_train['message']])
X_test_w2v = np.array([get_w2v_vector(text) for text in df_x_test['message']])

# 8. Chuẩn hóa đặc trưng
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_w2v)
X_test_scaled = scaler.transform(X_test_w2v)

# 9. Huấn luyện mô hình KNN
param_grid = {
    'n_neighbors': [17],
    'metric': ['cosine']
}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid.fit(X_train_scaled, y_encoded)

# 10. Dự đoán và đánh giá kết quả
y_pred = grid.best_estimator_.predict(X_test_scaled)

# 11. Đánh giá mô hình
print("Best KNN Parameters:", grid.best_params_)
print("Accuracy:", accuracy_score(df_y_test['category'], le.inverse_transform(y_pred)))
print("Classification Report:\n", classification_report(le.inverse_transform(df_y_test['category']), le.inverse_transform(y_pred)))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Best KNN Parameters: {'metric': 'cosine', 'n_neighbors': 17}
Accuracy: 0.9153664302600473
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.92      0.94      1566
           1       0.80      0.89      0.85       549

    accuracy                           0.92      2115
   macro avg       0.88      0.91      0.89      2115
weighted avg       0.92      0.92      0.92      2115

